## 6. Data Cleaning

Before proceeding with analysis, we perform comprehensive data cleaning to ensure data quality and validity. This section documents all cleaning steps and justifies why additional cleaning is or isn't necessary.

### 6.1 Ingredient Name Normalization (Already Completed)

**Purpose**: Standardize ingredient names to eliminate duplicates caused by different phrasings, spellings, and formatting.

**Method**: We utilized the pre-processed ingredient normalization table (from `ingredients.json`) that maps raw ingredient strings to canonicalized forms. This mapping was generated with the help of an LLM

**Impact**:
- Consolidated variants like "extra-virgin olive oil", "olive oil", "evoo" → `olive oil`
- Unified measurement variations: "2 cups all-purpose flour" → `flour`
- Standardized plurals and spellings

**Note**: This step was applied during the transformation phase (Section 5) when converting ingredient IDs to canonical names.

### 6.2 Check for Missing Values

In [2]:
import pandas as pd
df = pd.read_csv('dataset/prepared_recipes_raw.csv')

print("=" * 60)
print("MISSING VALUES CHECK")
print("=" * 60)

# Check for null values
missing_counts = df.isnull().sum()
missing_percentages = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Percentage': missing_percentages
})

print("\nMissing values by column:")
print(missing_df)

# Check for empty strings in ingredients
empty_ingredients = df[df['ingredients'] == ''].shape[0]
print(f"\nRecipes with empty ingredient lists: {empty_ingredients}")

# Check for zero ingredient counts
zero_count = df[df['ingredient_count'] == 0].shape[0]
print(f"Recipes with zero ingredients: {zero_count}")

print("\nNo missing values detected" if df.isnull().sum().sum() == 0 and empty_ingredients == 0 else "Missing values found")

MISSING VALUES CHECK

Missing values by column:
                  Missing Count  Percentage
recipe_id                     0         0.0
cuisine                       0         0.0
ingredients                   0         0.0
ingredient_count              0         0.0

Recipes with empty ingredient lists: 0
Recipes with zero ingredients: 0

No missing values detected


### 6.3 Check for Duplicate Recipes

In [ ]:
print("=" * 60)
print("DUPLICATE RECIPES CHECK")
print("=" * 60)

# Check for duplicate recipe IDs
duplicate_ids = df['recipe_id'].duplicated().sum()
print(f"\nDuplicate recipe IDs: {duplicate_ids}")

# Check for exact duplicate rows (same cuisine + ingredients)
duplicate_exact = df.duplicated(subset=['cuisine', 'ingredients']).sum()
print(f"Exact duplicate recipes (same cuisine + ingredients): {duplicate_exact}")

# Check for recipes with identical ingredients but different cuisines
# This is actually valid (same dish in different cuisines), not an error
duplicate_ingredients = df.duplicated(subset=['ingredients'], keep=False)
if duplicate_ingredients.any():
    print(f"\nRecipes with identical ingredients across cuisines: {duplicate_ingredients.sum()}")
    print("(This is valid - same ingredient set can appear in multiple cuisines)")
    
print("\nNo problematic duplicates" if duplicate_ids == 0 and duplicate_exact == 0 else "Duplicates found")

### 6.4 Validate Data Types and Ranges

In [ ]:
print("=" * 60)
print("DATA TYPE AND RANGE VALIDATION")
print("=" * 60)

print("\nColumn data types:")
print(df.dtypes)

# Validate ingredient_count matches actual count in ingredients string
print("\n" + "-" * 60)
print("Validating ingredient_count accuracy...")
print("-" * 60)

df['actual_count'] = df['ingredients'].apply(lambda x: len(x.split(',')) if x else 0)
mismatched = df[df['ingredient_count'] != df['actual_count']]

if len(mismatched) > 0:
    print(f"Found {len(mismatched)} recipes with mismatched counts")
    print(mismatched[['recipe_id', 'ingredient_count', 'actual_count']].head())
else:
    print("All ingredient counts are accurate")

# Check for unrealistic values
print("\n" + "-" * 60)
print("Ingredient count statistics:")
print("-" * 60)
print(df['ingredient_count'].describe())

# Flag suspiciously low or high counts (optional - for reporting only)
very_low = df[df['ingredient_count'] <= 2]
very_high = df[df['ingredient_count'] >= 30]
print(f"\nRecipes with <= 2 ingredients: {len(very_low)} ({len(very_low)/len(df)*100:.2f}%)")
print(f"Recipes with ≥30 ingredients: {len(very_high)} ({len(very_high)/len(df)*100:.2f}%)")
print("(These are valid but notable edge cases)")

# Drop temporary column
df.drop('actual_count', axis=1, inplace=True)

### 6.5 Check Cuisine Label Consistency

In [ ]:
print("=" * 60)
print("CUISINE LABEL VALIDATION")
print("=" * 60)

# Check for unusual characters or formatting issues
print(f"\nUnique cuisines: {df['cuisine'].nunique()}")
print("\nCuisine labels:")
for cuisine in sorted(df['cuisine'].unique()):
    count = len(df[df['cuisine'] == cuisine])
    print(f"  • {cuisine:20s} ({count:,} recipes)")

# Check for potential typos or inconsistencies (whitespace, case issues)
has_whitespace = df['cuisine'].str.contains(r'^\s|\s$', regex=True).any()
print(f"\nNo leading/trailing whitespace" if not has_whitespace else "Found whitespace issues")

# All cuisine labels should be lowercase (standard format for this dataset)
has_uppercase = df['cuisine'].str.contains(r'[A-Z]', regex=True).any()
print(f"All labels lowercase" if not has_uppercase else "Found uppercase letters")

### 6.6 Data Cleaning Summary and Justification

**Cleaning Steps Performed:**
1. **Ingredient Normalization** (pre-processing): Standardized all ingredient names using FoodOn ontology mapping
2. **Missing Value Check**: Verified no null values, empty ingredient lists, or zero counts
3. **Duplicate Detection**: Confirmed no duplicate recipe IDs or exact duplicate records
4. **Data Type Validation**: Verified correct types and validated ingredient_count accuracy
5. **Cuisine Label Validation**: Ensured consistent formatting and no typos in cuisine labels

**Why No Additional Cleaning is Necessary:**

The dataset is already high-quality and suitable for association mining because:

1. **Structured Source**: The original Kaggle dataset was curated for a machine learning competition with strict quality controls
2. **Graph Structure**: The hypergraph format (node IDs → ingredient names) prevents many common data entry errors
3. **No Noise in Transactions**: Each recipe is a valid transaction with meaningful ingredients
4. **No Outlier Removal Needed**: Even recipes with 2 ingredients or 30+ ingredients are valid culinary compositions (e.g., "salt and pepper" vs. complex stews). Removing these would bias the association mining results
5. **Complete Records**: No missing cuisines or ingredients - every recipe has complete information
6. **Semantic Normalization Already Applied**: The ingredient canonicalization resolved the primary data quality issue (lexical variation)

**For Association Mining Specifically:**
- We do NOT remove low-frequency ingredients (despite sparsity) because rare ingredients can form interesting association rules
- We do NOT filter recipes by ingredient count because basket size variability is part of the mining challenge
- We do NOT need to handle numerical ranges, outliers, or scaling - this is categorical transactional data

The dataset is now clean, validated, and ready for exploratory analysis and association mining.

## 7. Export Cleaned Dataset

Save the cleaned and validated dataset for use in EDA and association mining.

In [ ]:
import os

# Save cleaned dataset
output_path = 'dataset/prepared_recipes_cleaned.csv'
df.to_csv(output_path, index=False)

print("=" * 60)
print("CLEANED DATASET SAVED")
print("=" * 60)
print(f"Saved to: {output_path}")
print(f"Total records: {len(df):,}")
print(f"Columns: {list(df.columns)}")

# Get file size
file_size_bytes = os.path.getsize(output_path)
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"File size: {file_size_mb:.2f} MB ({file_size_bytes:,} bytes)")

print(f"\n✅ Dataset validation complete - ready for EDA and association mining!")
print(f"Next steps: Load this file in the EDA notebook for exploration and visualization.")